In [ ]:
#| default_exp debug

# DEBUG
> Detail API

In [ ]:
#|export
from avcv._imports import *

import mmcv
import pandas
import matplotlib


<string>:1: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
<string>:1: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
<string>:1: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [ ]:
# #| export
# @call_parse
# def make_mini_coco(json_path: Param(),
#                    image_prefix: Param(),
#                    out_dir: Param(),
#                    num_samples: Param("Num of sample",type=int) = 1000):
#     """
#         Helper function for creating a mini-dataset ensembles it's father
#     """
#     new_img_prefix = osp.join(out_dir, "images")

#     out_json = os.path.join(out_dir, "annotations", "mini_json.json")
#     if not osp.exists(out_json):
#         logger.info(f"Making mini dataset out_dir-> {out_dir}, num images:{num_samples}")
#         os.makedirs(os.path.join(out_dir, "images"), exist_ok=True)
#         os.makedirs(os.path.join(out_dir, "annotations"), exist_ok=True)
#         coco = COCO(json_path)
#         # imgs = coco.imgs
#         img_ids = list(coco.imgs.keys())
#         np.random.seed(0)
#         selected_img_ids = np.random.choice(img_ids, num_samples, replace=False)
#         imgs = coco.loadImgs(selected_img_ids)
#         selected_ann_ids = coco.getAnnIds(selected_img_ids)
#         anns = coco.loadAnns(selected_ann_ids)
#         for i, ann in enumerate(anns):
#             ann['iscrowd'] = False
#             anns[i] = ann
#         out_dict = dict(
#             images=imgs,
#             annotations=anns,
#             categories=coco.dataset['categories'],

#         )
#         for img in imgs:
#             path = osp.join(image_prefix, img['file_name'])
#             new_path = osp.join(new_img_prefix, img['file_name'])
#             mmcv.mkdir_or_exist(osp.dirname(new_path))
#             shutil.copy(path, new_path)

#         mmcv.dump(out_dict, out_json)
#     logger.info(f"{out_json}, {new_img_prefix}")
#     return out_json, new_img_prefix

In [ ]:
#| export
@call_parse
def dpython(cmd: Param(type=str)):
    for _ in range(3):
        cmd = cmd.replace('  ', '')
    i_split = cmd.index(".py")+4
    file = cmd[:i_split].strip().split(' ')[1]

    args = cmd[i_split:].split(' ')
    cfg_name = os.environ.get("DNAME", "Latest-Generated")
    cfg = {
        "name": f"Python: {cfg_name}",
        "type": "python",
        "request": "launch",
        "program": f"{file}",
        "console": "integratedTerminal",
        "args": args,
    }
    # pp(cfg)
    mmcv.mkdir_or_exist(".vscode")
    try:
        lauch = mmcv.load(".vscode/launch.json")
    except Exception as e:
        lauch = {
            "version": "0.2.0",
            "configurations": [

            ]
        }
        logger.warning(e)
        
    replace = False
    for i, _cfg in enumerate(lauch['configurations']):
        if _cfg["name"] == cfg["name"]:
            lauch["configurations"][i] = cfg
            replace = True
    if not replace:
        lauch["configurations"] += [cfg]
        mmcv.dump(lauch, '.vscode/launch.json')